In [39]:
%%capture
!pip install langchain-community pypdf
!pip install -qU langchain-huggingface
!pip install -U langchain-google-genai
!pip install "unstructured[image]"
!pip install pillow opencv-python

In [40]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()


··········


In [ ]:
from langchain_community.document_loaders import CSVLoader, PyPDFLoader

# List of files (you can mix CSV and PDF)
file_paths = [
    "",
     "" # upload your own file paths
]

all_documents = []

for path in file_paths:
    if path.endswith(".csv"):
        loader = CSVLoader(file_path=path)
    elif path.endswith(".pdf"):
        loader = PyPDFLoader(file_path=path)
    else:
        print(f"⚠️ Unsupported file format: {path}")
        continue

    docs = loader.load()
    all_documents.extend(docs)

# Your combined documents
print(all_documents)
all_docs = all_documents

[Document(metadata={'source': '/content/Hhld10_HouseholdFacility.csv', 'row': 0}, page_content='prov: 0\ndist: 0\ngapa: 0\nprovname: NEPAL\ndname: NEPAL\ngapaname: NEPAL\n: \nrowtotal: 6660841\nx_NoFacility: 258540\natleastOne: 6402301\na_Radio: 2280123\nb_TV: 3291185\nc_LandLinePhone: 301604\nd_MobilePhone1: 4876561\ne_MobilePhone2: 4862885\nf_Computer: 998478\ng_Internet: 2514510\nh_CarJeep: 204990\ni_Motorcycle: 1816121\nj_Bicycle: 2347433\nk_ElectricFan: 3535338\nl_Refrigerator: 1577195\nm_WashingMachine: 279637\nn_AirConditionr: 56601\nLevel: National'), Document(metadata={'source': '/content/Hhld10_HouseholdFacility.csv', 'row': 1}, page_content='prov: 1\ndist: 0\ngapa: 0\nprovname: PROVINCE - 1\ndname: PROVINCE - 1\ngapaname: PROVINCE - 1\n: \nrowtotal: 1190755\nx_NoFacility: 44351\natleastOne: 1146404\na_Radio: 443014\nb_TV: 649594\nc_LandLinePhone: 36632\nd_MobilePhone1: 864107\ne_MobilePhone2: 883723\nf_Computer: 140580\ng_Internet: 453858\nh_CarJeep: 25960\ni_Motorcycle: 294

In [42]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=400, add_start_index=True
)
all_splits = text_splitter.split_documents(all_docs)

print(len(all_splits))

937


In [43]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="minishlab/potion-base-8M")

In [44]:
%%capture
!pip install tqdm

In [45]:
from tqdm import tqdm

all_vectors = []
for doc in tqdm(all_splits, desc="Generating embeddings"):
    vec = embeddings.embed_query(doc.page_content)
    all_vectors.append(vec)

print(f"\n✅ Generated {len(all_vectors)} embeddings.")

Generating embeddings: 100%|██████████| 937/937 [00:01<00:00, 711.82it/s]


✅ Generated 937 embeddings.


In [47]:
%%capture
!pip install -qU langchain-chroma

In [ ]:
import chromadb

cloud_client = chromadb.CloudClient(
    api_key="",
    tenant="",
)

In [49]:
from langchain_chroma import Chroma

vector_store = Chroma(
    client=cloud_client,
    collection_name="Pepsi",
    embedding_function=embeddings
)

In [50]:
# Add documents in batches
batch_size = 300
ids = [] # Initialize an empty list to store all the ids
for i in range(0, len(all_splits), batch_size):
    batch = all_splits[i : i + batch_size]
    batch_ids = vector_store.add_documents(documents=batch)
    ids.extend(batch_ids) # Extend the ids list with the ids from the current batch
    print(f"Added batch {i // batch_size + 1}")

print("All documents added.")

Added batch 1
Added batch 2
Added batch 3
Added batch 4
All documents added.


In [51]:
collection = cloud_client.get_or_create_collection("Pepsi")

In [ ]:
import os
from langchain.chat_models import init_chat_model

os.environ["GOOGLE_API_KEY"] = ""

model = init_chat_model("google_genai:gemini-2.5-flash-lite")

In [54]:
%%capture
!pip install ddgs

In [ ]:
from langchain.tools import tool
from ddgs import DDGS


@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """Retrieve information to help answer a query."""
    retrieved_docs = vector_store.similarity_search(query, k=1)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs


@tool("web_search", return_direct=False)
def web_search_tool(query: str, max_results: int = 10) -> str:
    """
    Perform a real web search using DuckDuckGo.
    Returns formatted results for LLM or pipeline use.
    """
    results = []
    with DDGS() as ddgs:
        for r in ddgs.text(query, max_results=max_results):
            results.append({
                "title": r.get("title"),
                "link": r.get("href"),
                "snippet": r.get("body")
            })

    if not results:
        return "No results found."

    formatted = "\n\n".join(
        [f"🔹 {r['title']}\n{r['snippet']}\nURL: {r['link']}" for r in results]
    )
    return formatted


@tool("web_image_link_tool", return_direct=False)
def web_image_link_tool(query: str, max_results: int = 10) -> str:
    """
    Perform a real web search using DuckDuckGo.
    Always returns both text results and related images (for Colab use).

    Args:
        query (str): Search query
        max_results (int): Number of text results
    """
    results_text = []
    results_images = []

    with DDGS() as ddgs:
        # --- Text search ---
        for r in ddgs.text(query, max_results=max_results):
            results_text.append({
                "title": r.get("title"),
                "link": r.get("href"),
                "snippet": r.get("body")
            })

        # --- Image search (always performed) ---
        for r in ddgs.images(query, max_results=min(5, max_results)):
            results_images.append({
                "title": r.get("title"),
                "image_url": r.get("image"),
                "thumbnail": r.get("thumbnail"),
                "page_url": r.get("url")
            })

    # --- Handle no results ---
    if not results_text and not results_images:
        return "No results found."

    # --- Build HTML for both ---
    html_content = "<h3>🔎 Web Results:</h3>"
    if results_text:
        for r in results_text:
            html_content += f"""
            <div style='margin-bottom:10px;'>
                <b>{r['title']}</b><br>
                {r['snippet']}<br>
                <a href="{r['link']}" target="_blank">{r['link']}</a>
            </div>
            """
    else:
        html_content += "<p>No text results found.</p>"

    # --- Add image section ---
    html_content += "<h3>🖼️ Related Images:</h3>"
    if results_images:
        for r in results_images:
            html_content += f"""
            <div style='margin:10px; display:inline-block; text-align:center;'>
                <img src="{r['thumbnail']}" width="200"><br>
                <a href="{r['image_url']}" target="_blank">{r['title']}</a>
            </div>
            """
    else:
        html_content += "<p>No images found.</p>"

    # --- Display inline in Colab ---
    display(HTML(html_content))
    return "✅ Web and image results displayed above 👆"


In [56]:
%%capture
!pip install -U langchain langchain-community langchain-core

In [ ]:
from langchain.agents import create_agent
from langchain_core.language_models.chat_models import BaseChatModel


tools = [retrieve_context, web_search_tool]
# If desired, specify custom instructions
prompt = (


          """
You are a **Research Helper Agent** that assists users with academic and scientific queries.

The user will specify whether they want:
- **Context Retrieval** → to analyze or summarize information from their uploaded research papers, or
- **Web Search** → to explore external academic sources from the internet.

Your job:
1. Use the tool the user selected.
2. Generate a clear, factual, and well-organized academic-style answer.
3. If using context retrieval, cite from uploaded papers (e.g., “According to the uploaded paper…”).
4. If using web search, summarize from the found web content without fabricating citations.

Keep answers:
- Concise yet informative
- Objective and research-oriented
- Avoid personal opinions or speculation
"""

)
agent = create_agent(model, tools, system_prompt=prompt)

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break

    # ✅ Ask user whether to use web search
    use_web = input("Use web search? (y/n): ").strip().lower()

    if use_web == "y":
        tool_name = "web_search"
        print("🔎 Using web search tool...")
    else:
        tool_name = "retrieve_context"
        print("📘 Using local context retrieval...")

    # Run only the selected tool
    tool = next(t for t in tools if t.name == tool_name)
    tool_output = tool.invoke({"query": query})

    # Send retrieved info + user query to LLM
    input_messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": query},
        {"role": "assistant", "content": tool_output},
    ]

    # Stream the response
    for event in agent.stream({"messages": input_messages}, stream_mode="values"):
        event["messages"][-1].pretty_print()


You: explain me the hardware of Ecg
Use web search? (y/n): y
🔎 Using web search tool...
================================== Ai Message ==================================

🔹 Holter monitor - Wikipedia
In medicine, a Holter monitor is a type of ambulatory electrocardiography device, a portable device for cardiac monitoring worn for at least 24 hours. The Holter's most common use is for monitoring ECG heart activity.
URL: https://en.wikipedia.org/wiki/Holter_monitor

🔹 The remote health monitoring system ( hardware development and...)
This section contains the study of ECG hardware performance in real-life. First, the hardware performance would be demonstrated understanding still, walk and run conditions. Followed by the performance test on the heart rate calculation. 3.1.1 ECG sensing function.
URL: https://www.rs-online.com/designspark/the-remote-health-monitoring-system-hardware-development-and-measurement

🔹 (PDF) Electrocardiograph ( ECG ) Circuit Design and Software-based...
Since th